
# Объектная модель Python II
1. Где реально хранятся атрибуты: `obj.__dict__`, `type(obj).__dict__`
2. Механизм поиска атрибутов: порядок и приоритеты (экземпляр → класс → базовые классы)
3. `getattr`, `setattr`, `hasattr` как инструменты работы с объектной моделью
4. Отличия `__getattribute__` и `__getattr__` (на уровне применения)
5. `__slots__`: зачем нужен, ограничения и эффект на память/производительность
6. Типовые ошибки: неожиданные атрибуты, опечатки, «тихие» баги без `__slots__`

## Перед началом: что такое класс (напоминание)

### Что такое класс
В Python класс — это одновременно:
- **шаблон** для создания объектов;
- **пространство имён** для атрибутов (методы, константы, свойства);
- **модель поведения** (набор операций/правил).

```python
class User:
    pass

u = User()  # экземпляр класса
```

### Класс — это объект
Класс тоже является объектом и создаётся метаклассом `type`:

```python
class A:
    pass

type(A)  # <class 'type'>
```

Следствия:
- класс можно передавать как значение;
- у класса есть атрибуты (`A.__dict__`, `A.mro()`);
- класс участвует в поиске атрибутов и связывании методов.

### Основные характеристики класса
1. **Пространство имён класса**: `Class.__dict__` (методы, атрибуты, дескрипторы)
2. **Связь с экземплярами**: экземпляр хранит состояние (обычно `obj.__dict__`)
3. **Наследование и MRO**: определяют порядок поиска атрибутов
4. **Метакласс**: механизм создания класса (важно для фреймворков)

In [ ]:
class A:
    class_attr = 10

    def __init__(self, instance_attr: int):
      self.instance_attr = instance_attr

    def method(self):
        return 'ok'

obj = A(instance_attr=10)
obj.class_attr = 20

print('type(A):', type(A))
print('A.__dict__ has method:', 'method' in A.__dict__)
print('A.__dict__:',A.__dict__)
print('obj.__dict__:', obj.__dict__)
print('A.class_attr:', A.class_attr)
print('obj.class_attr:', obj.class_attr, '(доступ через класс)')


type(A): <class 'type'>
A.__dict__ has method: True
A.__dict__: {'__module__': '__main__', 'class_attr': 10, '__init__': <function A.__init__ at 0x7942ed191300>, 'method': <function A.method at 0x7942ed1911c0>, '__dict__': <attribute '__dict__' of 'A' objects>, '__weakref__': <attribute '__weakref__' of 'A' objects>, '__doc__': None}
obj.__dict__: {'instance_attr': 10, 'class_attr': 20}
A.class_attr: 10
obj.class_attr: 20 (доступ через класс)


## Где это используется в production

Понимание объектной модели — не «теория ради теории». Эти механизмы лежат в основе популярных библиотек и фреймворков.

### Примеры
- **Django ORM**: поля модели — дескрипторы; метаклассы формируют модель, админку и запросы.
- **Pydantic / FastAPI**: модели данных, валидация, управляемое присваивание атрибутов.
- **SQLAlchemy**: ленивые атрибуты, прокси, дескрипторы для колонок и отношений.
- **dataclasses**: генерация кода класса и управление полями.
- **`__slots__`**: экономия памяти и защита от «тихих» багов в больших моделях/пайплайнах.

### Что даёт знание объектной модели backend-разработчику
- читать и понимать код фреймворков;
- проектировать расширяемые системы (плагины, обработчики, динамические диспетчеры);
- ловить скрытые ошибки (опечатки, неожиданные атрибуты, перехват доступа);
- осознанно оптимизировать память и поведение объектов.


## 0. Что такое «атрибут» в Python?

В Python «атрибут» — это **имя**, которое резолвится в некоторое **значение** при обращении через точку:

```python
obj.attr
```

Важно:
- атрибуты экземпляра и атрибуты класса — разные «слои»;
- поиск атрибутов — **алгоритм**, а не «магия»;
- механизмы `__getattribute__`, `__getattr__`, дескрипторы, `__slots__` влияют на этот алгоритм.


In [ ]:
from pprint import pprint
import sys
import types

## 1. Где реально хранятся атрибуты



### 1.1. `obj.__dict__`
У большинства «обычных» объектов атрибуты экземпляра хранятся в словаре `obj.__dict__`.

- Ключи — строки (имена атрибутов)
- Значения — любые объекты

> Но `__dict__` может отсутствовать (например, при использовании `__slots__`).

In [ ]:
class User:
    def __init__(self, name: str):
        self.name = name
        self.is_active = True

u = User('Samir')
u.b = 12
u.__dict__["bal"] = 12
print(u.name)
print(u.__dict__)


Samir
{'name': 'Samir', 'is_active': True, 'b': 12, 'bal': 12}


### 1.2. `type(obj).__dict__`
Атрибуты **класса** (методы, константы, свойства и т.п.) находятся в `Class.__dict__`.

Важно: `Class.__dict__` — это не обычный `dict`, а **mappingproxy** (read-only view), чтобы защищать внутреннее состояние класса

In [ ]:
class User:
    role = 'student'  # атрибут класса

    def greet(self):
        return f"Hi, I'm {self.name}"

u = User()
print('instance dict:', u.__dict__)
print('class dict type:', type(User.__dict__))
print('role in class dict:', 'role' in User.__dict__)
print('greet in class dict:', 'greet' in User.__dict__)

try:
    User.__dict__['x'] = 1
except TypeError as e:
    print('cannot modify mappingproxy:', e)


instance dict: {}
class dict type: <class 'mappingproxy'>
role in class dict: True
greet in class dict: True
cannot modify mappingproxy: 'mappingproxy' object does not support item assignment


### 1.3. Экземпляр vs класс: одинаковые имена
Если у экземпляра и класса есть атрибут с одним и тем же именем, **приоритет у экземпляра**.

Это одна из причин «тихих» багов: вы можете случайно «перекрыть» атрибут класса атрибутом экземпляра.


In [ ]:
class Config:
    timeout = 5  # секунды

c = Config()
print('Before:', c.timeout, '(from class)')

c.timeout = 999  # теперь это атрибут экземпляра
print('After:', c.timeout, '(from instance)')
print('Instance dict:', c.__dict__)
print('Class timeout still:', Config.timeout)


Before: 5 (from class)
After: 999 (from instance)
Instance dict: {'timeout': 999}
Class timeout still: 5


## 2. Механизм поиска атрибутов: порядок и приоритеты


Упрощённо, для **обычного** атрибута `obj.attr` Python делает примерно так:

1. Вызов `type(obj).__getattribute__(obj, 'attr')` (всегда)
2. Внутри алгоритма:
   - проверка **дескрипторов** в классе (например, `property`)
   - поиск в `obj.__dict__` (если он есть)
   - поиск в `type(obj).__dict__`
   - поиск по базовым классам согласно **MRO** (Method Resolution Order)

Если быть точнее:

1.	Вызов type(obj).__getattribute__(obj, "attr").
2.	Внутри __getattribute__ поиск происходит в порядке:
  - data descriptor в классе или его MRO (__set__ или __delete__)
  - атрибут в obj.__dict__ (если есть)
  - non-data descriptor или обычный атрибут в классе/MRO
  - вызов __getattr__, если атрибут не найден
  - иначе AttributeError


In [ ]:
class A:
  def __init__(self) -> None:
      self._password = "1234"


  @property
  def password(self):
    return hash(self._password)


  @password.setter
  def password(self, value):
    if len(value) < 10:
      self._password = value
    else:
      raise Exception("Too long password")


a = A()

print(a.password)

a.password = "avx"

print(a.password)

-1887162208766598736
8580896131856238979



### 2.1. MRO (порядок разрешения методов)
MRO — это линейный порядок классов, который Python использует для поиска атрибутов по наследованию.

In [ ]:
class A:
    x = 'A.x'

class B(A):
    pass

class C(A):
    x = 'C.x'

class D(B, C):
    pass

print('MRO for D:', [cls.__name__ for cls in D.mro()])
print('D.x:', D.x)

MRO for D: ['D', 'B', 'C', 'A', 'object']
D.x: C.x


### 2.2. Экземпляр → класс → базовые классы
Сначала Python пытается найти значение на экземпляре, затем в классе, затем по базовым классам.

Сделаем это наблюдаемым.


In [ ]:
class Base:
    value = 'from Base'

class Child(Base):
    value = 'from Child'

obj = Child()
print(obj.value)  # из класса Child

obj.value = 'from instance'
print(obj.value)  # теперь из экземпляра

del obj.value
print(obj.value)  # снова из класса Child


from Child
from instance
from Child


### 2.3. Важно: дескрипторы могут изменить приоритеты
Например, `property` (data descriptor) имеет приоритет **выше**, чем `obj.__dict__`.

То есть `obj.__dict__['x'] = ...` не обязательно переопределит `obj.x`, если `x` — `property`.


In [ ]:
class P:
    def __init__(self):
        self._x = 10

    @property
    def x(self):
        return self._x

p = P()
print('p.x:', p.x)

p.__dict__['x'] = 999
print('p.__dict__:', p.__dict__)
print('p.x still:', p.x, '(property wins)')

p.x: 10
p.__dict__: {'_x': 10, 'x': 999}
p.x still: 10 (property wins)


## 3. `getattr`, `setattr`, `hasattr` как инструменты работы с объектной моделью

Эти функции работают поверх того же механизма, что и доступ через точку.

- `getattr(obj, name[, default])` — получить атрибут по строковому имени
- `setattr(obj, name, value)` — установить атрибут по строковому имени
- `hasattr(obj, name)` — проверить наличие атрибута



### 3.1. Когда это нужно
- динамическая маршрутизация (например, вызов обработчика по имени)
- плагины
- сериализация/десериализация
- конфигурации

In [ ]:
class Handler:
    def on_create(self):
        return 'created'

    def on_delete(self):
        return 'deleted'

h = Handler()

event = 'create'
method_name = f'on_{event}'

print('hasattr:', hasattr(h, method_name))
print('getattr call:', getattr(h, method_name)())

print('unknown event:', getattr(h, 'on_update', lambda: 'noop')())


hasattr: True
getattr call: created
unknown event: noop


In [ ]:
hasattr(h, "on_update")

False

In [ ]:
getattr(h, "on_update", "Handler object has no attribute on_update")

'Handler object has no attribute on_update'

### 3.2. Осторожно с `hasattr`
`hasattr(obj, name)` внутри делает `getattr` и **ловит `AttributeError`**.

Если ваш `__getattribute__` или `property` может бросать `AttributeError` по другой причине,
`hasattr` может вернуть `False` даже когда проблема не в отсутствии атрибута.


In [ ]:
class Weird:
    @property
    def x(self):
        raise AttributeError('not missing, but failing')

w = Weird()
print('hasattr(w, "x"):', hasattr(w, 'x'))
try:
    print(w.x)
except AttributeError as e:
    print('direct access error:', e)


hasattr(w, "x"): False
direct access error: not missing, but failing


## 4. `__getattribute__` vs `__getattr__`

Оба метода связаны с доступом к атрибутам, но вызываются в разных ситуациях.

### 4.1. `__getattribute__(self, name)`
- вызывается **всегда** при доступе к атрибуту `obj.name`
- если вы его переопределяете — вы берёте ответственность за весь поиск атрибутов


### 4.2. `__getattr__(self, name)`
- вызывается **только если атрибут не найден** обычным способом
- удобен для:
  - вычисляемых «виртуальных» атрибутов
  - прокси/обёрток
  - обратной совместимости

Главное правило: **не сломайте доступ к базовым атрибутам**.

In [ ]:
class OnlyGetattr:
    def __init__(self):
        self.real = 123

    def __getattribute__(self, name: str, /):
        print(f"GetAttribute was called for name: {name}")
        return f"getattribute__{super().__getattribute__(name)}"

    def __getattr__(self, name):
        if name.startswith('virtual_'):
            return f'computed:{name}'
        raise AttributeError(name)

og = OnlyGetattr()
print('real:', og.real)
print('virtual:', og.virtual_test)

try:
    print(og.missing)
except AttributeError as e:
    print('missing:', e)

GetAttribute was called for name: real
real: getattribute__123
GetAttribute was called for name: virtual_test
virtual: computed:virtual_test
GetAttribute was called for name: missing
missing: missing


### 4.3. Пример `__getattribute__`: логирование доступа

Переопределяя `__getattribute__`, важно использовать `object.__getattribute__(self, name)`
для доступа к реальному механизму, иначе легко получить бесконечную рекурсию.

In [ ]:
class LoggingAccess:
    def __init__(self):
        self.x = 10

    def __getattribute__(self, name):
        print(f'[LOG] access: {name}')
        return object.__getattribute__(self, name)

la = LoggingAccess()
print(la.x)


[LOG] access: x
10


In [ ]:
class Bad:
    def __init__(self):
        self.x = 1

    def __getattribute__(self, name):
        return self.name

b = Bad()
print(b.x)

RecursionError: maximum recursion depth exceeded

## 5. `__slots__`: зачем нужен, ограничения и эффект

### 5.1. Что делает `__slots__`
`__slots__` позволяет **запретить создание `__dict__` у экземпляров** и ограничить набор допустимых атрибутов.

Это даёт:
- меньше памяти на объект (нет динамического словаря)
- иногда быстрее доступ к атрибутам (за счёт более прямого хранения)
- защиту от опечаток и неожиданных атрибутов

### 5.2. Ограничения
- нельзя добавлять новые атрибуты, не перечисленные в `__slots__`
- при наследовании нужно аккуратно дополнять `__slots__`
- если нужны «динамические атрибуты», `__slots__` может мешать

> `__slots__` — инструмент оптимизации и дисциплины модели, а не «обязательная практика всегда».

#### 5.2.1. Практика: как запретить добавлять новые атрибуты

В production чаще всего хотят **два эффекта**:

1) **Экономия памяти** (у объектов нет динамического `__dict__`),
2) **Защита модели** от «тихих» багов (опечатки и неожиданные поля).

Самый простой способ запретить добавлять новые атрибуты — использовать `__slots__` **без** `__dict__`.


In [ ]:
# Запрет на новые атрибуты через __slots__
class UserSlotsStrict:
    __slots__ = ('name', 'email')

    def __init__(self, name: str, email: str):
        self.name = name
        self.email = email

u = UserSlotsStrict('Samir', 's@example.com')
print('name:', u.name)
print('has __dict__:', hasattr(u, '__dict__'))

# Опечатка или неожиданное поле → AttributeError
try:
    u.emial = 'typo@example.com'  # опечатка
except AttributeError as e:
    print('Caught:', e)

try:
    u.age = 30  # новое поле
except AttributeError as e:
    print('Caught:', e)

name: Samir
has __dict__: False
Caught: 'UserSlotsStrict' object has no attribute 'emial'
Caught: 'UserSlotsStrict' object has no attribute 'age'


### 5.2.2. `__dict__` внутри `__slots__`: когда это нужно

Иногда в production хотят **частично** ограничить модель:
- основные поля фиксированы (в `__slots__`),
- но иногда нужно хранить произвольные расширения (feature flags, метаданные, отладочные поля).

Для этого в `__slots__` можно **явно добавить** строку `"__dict__"`.

Тогда:
- объект будет иметь `__dict__`;
- можно будет добавлять любые дополнительные атрибуты;
- но слотовые атрибуты будут храниться «компактно».

Это компромисс: защита от опечаток хуже, но гибкость выше.


In [ ]:
# __slots__ + '__dict__' = слоты + возможность динамических атрибутов
class UserSlotsWithDict:
    __slots__ = ('name', 'email', '__dict__')

    def __init__(self, name: str, email: str):
        self.name = name
        self.email = email

u = UserSlotsWithDict('Samir', 's@example.com')
print('has __dict__:', hasattr(u, '__dict__'))

# Теперь новые атрибуты разрешены
u.age = 30
u.debug_tag = 'DEV'
print('u.__dict__:', u.__dict__)

# Но опечатка снова станет "тихим" багом
u.emial = 'typo@example.com'
print('typo stored:', u.__dict__.get('emial'))
print('email still:', u.email)


has __dict__: True
u.__dict__: {'age': 30, 'debug_tag': 'DEV'}
typo stored: typo@example.com
email still: s@example.com


### 5.2.3 Как запретить добавление новых атрибутов без `__slots__`

Бывает, что по архитектуре вы **не можете** использовать `__slots__` (например, динамическое создание полей, прокси-объекты).
Тогда можно контролировать присваивание через `__setattr__`.

Идея: разрешаем только заранее известный набор имён, всё остальное — ошибка.

Это используется в production реже, чем `__slots__`, но полезно знать.


In [ ]:
class StrictSetattr:
    _allowed = {'name', 'email'}

    def __init__(self, name: str, email: str):
        self.name = name
        self.email = email

    def __setattr__(self, name, value):
        if name not in self._allowed and not name.startswith('_'):
            raise AttributeError(f'Unknown attribute: {name}')
        super().__setattr__(name, value)

s = StrictSetattr('Samir', 's@example.com')
print(s.__dict__)

try:
    s.age = 30
except AttributeError as e:
    print('Caught:', e)

{'name': 'Samir', 'email': 's@example.com'}
Caught: Unknown attribute: age


### 5.2.4 `__slots__` и наследование

Если вы используете `__slots__` в базовом классе и хотите добавить поля в наследнике,
то наследник должен объявить **свои** `__slots__`.

Важно: `__slots__` **не наследуются как список**, они комбинируются механизмом реализации.


In [ ]:
class BaseSlots:
    __slots__ = ('id',)
    def __init__(self, id: int):
        self.id = id

class ChildSlots(BaseSlots):
    __slots__ = ('name',) #$tRo4kA_v_$k0bKaX
    def __init__(self, id: int, name: str):
        super().__init__(id)
        self.name = name

c = ChildSlots(1, 'x')
print(c.id, c.name)

try:
    c.extra = 1
except AttributeError as e:
    print('Caught:', e)


1 x
Caught: 'ChildSlots' object has no attribute 'extra'


In [ ]:
class WithDict:
    def __init__(self, x):
        self.x = x
        self.y = x + 1

class WithSlots:
    __slots__ = ('x', 'y')
    def __init__(self, x):
        self.x = x
        self.y = x + 1

wd = WithDict(1)
ws = WithSlots(1)

print('WithDict has __dict__:', hasattr(wd, '__dict__'))
print('WithSlots has __dict__:', hasattr(ws, '__dict__'))

try:
    ws.z = 123
except AttributeError as e:#ssh://git@git-ssh.21-school.ru:2222/students_repo/wareluci/D02T02.ID_1577482-1.git
    print('cannot set ws.z:', e)


WithDict has __dict__: True
WithSlots has __dict__: False
cannot set ws.z: 'WithSlots' object has no attribute 'z'


### 5.3. Память

`sys.getsizeof` показывает размер *объекта-обёртки*, но не всегда учитывает связанные структуры.
Однако сравнение полезно, если делать его на большом количестве объектов.


In [ ]:
import sys

N = 50_000

objs_dict = [WithDict(i) for i in range(N)]
objs_slots = [WithSlots(i) for i in range(N)]

size_dict = sys.getsizeof(objs_dict) + sum(sys.getsizeof(o) for o in objs_dict) + sum(sys.getsizeof(o.__dict__) for o in objs_dict)
size_slots = sys.getsizeof(objs_slots) + sum(sys.getsizeof(o) for o in objs_slots)

print(f'Approx total size WithDict  : {size_dict/1024/1024:.2f} MB')
print(f'Approx total size WithSlots : {size_slots/1024/1024:.2f} MB')
print('Note: numbers are approximate (Python allocator & internals).')

Approx total size WithDict  : 6.91 MB
Approx total size WithSlots : 2.71 MB
Note: numbers are approximate (Python allocator & internals).


In [ ]:
# __slots__ + '__dict__' = слоты + возможность динамических атрибутов
class UserSlotsWithDict:
    __slots__ = ('x', '__dict__')

    def __init__(self, x):
        self.x = x
        self.y = x + 1

In [ ]:
import sys

N = 50_000

objs_dict = [WithDict(i) for i in range(N)]
objs_slots = [WithSlots(i) for i in range(N)]
objs_slots_dict = [UserSlotsWithDict(i) for i in range(N)]

size_dict = sys.getsizeof(objs_dict) + sum(sys.getsizeof(o) for o in objs_dict) + sum(sys.getsizeof(o.__dict__) for o in objs_dict)
size_slots = sys.getsizeof(objs_slots) + sum(sys.getsizeof(o) for o in objs_slots)
size_slots_dict = sys.getsizeof(objs_slots_dict) + sum(sys.getsizeof(o) for o in objs_slots_dict ) + + sum(sys.getsizeof(o.__dict__) for o in objs_slots_dict)

print(f'Approx total size WithDict  : {size_dict/1024/1024:.2f} MB')
print(f'Approx total size WithSlots : {size_slots/1024/1024:.2f} MB')
print(f'Approx total size WithSlots : {size_slots_dict/1024/1024:.2f} MB')

print('Note: numbers are approximate (Python allocator & internals).')

Approx total size WithDict  : 6.91 MB
Approx total size WithSlots : 2.71 MB
Approx total size WithSlots : 6.91 MB
Note: numbers are approximate (Python allocator & internals).


### 5.4. Производительность: микробенчмарк (осторожно!)

Скорость зависит от версии Python, CPU и окружения.
Цель — увидеть тенденцию, а не получить абсолютные цифры.


In [ ]:
import timeit

wd = WithDict(1)
ws = WithSlots(1)
wsd = UserSlotsWithDict(1)

N = 500_000_0
print('WithDict attr get:', timeit.timeit('WithDict(1).x', globals=globals(), number=500_000_0))

WithDict attr get: 0.636212353000019


In [ ]:
print('WithSlots attr get:', timeit.timeit('WithSlots(1).x', globals=globals(), number=500_000_0))
print('WithSlotsDict attr get X:', timeit.timeit('UserSlotsWithDict(1).x', globals=globals(), number=500_000_0))

WithSlots attr get: 0.5571863149999672
WithSlotsDict attr get X: 0.6379808149999917


In [ ]:
wsd_1 = UserSlotsWithDict(1)


In [ ]:
wsd_1.y = 1
print('WithSlotsDict attr get Y:', timeit.timeit('wsd_1.y', globals=globals(), number=500_000_000))

WithSlotsDict attr get Y: 6.913710908000098


## 6. Типовые ошибки и «тихие» баги без `__slots__`

### 6.1. Опечатки создают новые атрибуты
Если объект имеет `__dict__`, то **любая опечатка** создаст новый атрибут — и программа может продолжить работать, но неправильно.


In [ ]:
class TaskNoSlots:
    def __init__(self):
        self.status = 'new'

t = TaskNoSlots()

t.stauts = 'done'  # опечатка
print('t.status:', t.status)
print('t.__dict__:', t.__dict__)
print('Bug: status did not change!')

t.status: new
t.__dict__: {'status': 'new', 'stauts': 'done'}
Bug: status did not change!


### 6.2. Как `__slots__` помогает ловить такие ошибки


In [ ]:
class TaskSlots:
    __slots__ = ('status')
    def __init__(self):
        self.status = 'new'

ts = TaskSlots()
try:
    ts.stauts = 'done'  # опечатка
except AttributeError as e:
    print('Caught typo:', e)
print('ts.status:', ts.status)


Caught typo: 'TaskSlots' object has no attribute 'stauts'
ts.status: new


### 6.3. Неожиданные атрибуты и «поломка» инвариантов

Без дисциплины модели кто-то может «подложить» новый атрибут и обойти ваш API.
`__slots__` — дополнительный слой защиты: нельзя добавить лишнее.


In [ ]:
class Account:
    def __init__(self):
        self._balance = 0

    @property
    def balance(self):
        return self._balance

    def deposit(self, amount: int):
        if amount <= 0:
            raise ValueError('amount must be positive')
        self._balance += amount

acc = Account()
acc.deposit(10)
print('balance:', acc.balance)

acc.balnce = 1_000_000  # опечатка, но выглядит правдоподобно
print('acc.__dict__:', acc.__dict__)
print('balance still:', acc.balance)


balance: 10
acc.__dict__: {'_balance': 10, 'balnce': 1000000}
balance still: 10


## Практические примеры

### 1. Ленивые вычисления: @cached_property (кэш на объекте)

Пример: тяжёлый расчёт/запрос выполняется только при первом обращении, результат сохраняется в объекте

In [ ]:
from __future__ import annotations

import time
from dataclasses import dataclass
from functools import cached_property

@dataclass
class Report:
    user_id: int

    def _load_user_profile_from_db(self) -> dict:
        # имитация дорогого I/O (DB / HTTP)
        time.sleep(3)
        return {"id": self.user_id, "name": "Samir", "role": "admin"}

    @cached_property
    def user_profile(self) -> dict:
        # вычислится один раз, дальше берётся из obj.__dict__
        return self._load_user_profile_from_db()

r = Report(42)

t0 = time.time()
print(r.user_profile)  # медленно: "грузим" один раз
print(f"first access: {time.time() - t0:.3f}s")

t1 = time.time()
print(r.user_profile)  # быстро: уже кэшировано
print(f"second access: {time.time() - t1:.3f}s")

# Можно сбросить кэш (иногда нужно при инвалидировании)
del r.__dict__["user_profile"]
print("cache dropped")
print(r.user_profile)  # снова "медленно"

{'id': 42, 'name': 'Samir', 'role': 'admin'}
first access: 3.001s
{'id': 42, 'name': 'Samir', 'role': 'admin'}
second access: 0.000s
cache dropped
{'id': 42, 'name': 'Samir', 'role': 'admin'}


Где это похоже на прод:
- ORM/DAO кэшируют вычислимые свойства (например, вычисляемые поля, парсинг JSON из БД, “дорогие” агрегаты).
- В сервисах: ленивое получение токена, конфигурации, профиля пользователя, схемы и т.п.


### 2. Прокси-объект с ленивой инициализацией (Lazy Proxy)

Прокси «притворяется» объектом, но реальный объект создаёт/грузит только когда он действительно понадобился. Паттерн близок к lazy-load прокси в ORM.

In [ ]:
from __future__ import annotations

from typing import Callable, Generic, Optional, TypeVar

T = TypeVar("T")

class LazyProxy(Generic[T]):
    """
    Ленивый прокси: создаёт реальный объект только при первом доступе.
    Делегирует атрибуты/вызовы реальному объекту.
    """
    def __init__(self, factory: Callable[[], T]):
        self._factory = factory
        self._obj: Optional[T] = None

    def _get(self) -> T:
        if self._obj is None:
            self._obj = self._factory()
        return self._obj

    def __getattr__(self, name: str):
        # вызывается только если атрибут не найден на самом прокси
        return getattr(self._get(), name)

    def __repr__(self) -> str:
        if self._obj is None:
            return "<LazyProxy (not loaded)>"
        return f"<LazyProxy loaded obj={self._obj!r}>"

class User:
    def __init__(self, user_id: int):
        print(f"[DB] Loading user {user_id} ...")
        self.id = user_id
        self.name = "Samir"
        self.permissions = {"read", "write", "admin"}

    def can(self, perm: str) -> bool:
        return perm in self.permissions

# фабрика загрузки (DB/HTTP/кэш)
def load_user() -> User:
    return User(42)

u = LazyProxy(load_user)

print(u)             # ещё не грузили
print(u.name)        # тут произойдёт загрузка
print(u.can("admin"))  # дальше работает как обычный объект
print(u)             # уже loaded

<LazyProxy (not loaded)>
[DB] Loading user 42 ...
Samir
True
<LazyProxy loaded obj=<__main__.User object at 0x7e7807a67110>>


Где это похоже на прод:
- ORM (SQLAlchemy/Django) могут «откладывать» загрузку связанных сущностей/полей.
- Клиенты внешних сервисов: объект-сессия/клиент создаётся при первом реальном запросе.
- Большие модели в пайплайнах: объект “контекста” тянет тяжёлые ресурсы только по требованию.

### 3. Safe Lazy Proxy: запрет новых атрибутов + защита от опечаток

Идея: у прокси нет __dict__, поэтому proxy.nmae = ... не создаст новый атрибут и не замаскирует ошибку. Разрешим менять только внутренние поля _factory/_obj.

In [ ]:
from __future__ import annotations

from typing import Callable, Generic, Optional, TypeVar

T = TypeVar("T")

class SafeLazyProxy(Generic[T]):
    """
    - Лениво создаёт объект при первом реальном доступе.
    - Не имеет __dict__ (через __slots__) => нельзя "тихо" добавить новое поле.
    - __setattr__ запрещает любые атрибуты, кроме служебных.
    """
    __slots__ = ("_factory", "_obj")

    def __init__(self, factory: Callable[[], T]):
        object.__setattr__(self, "_factory", factory)
        object.__setattr__(self, "_obj", None)

    def _get(self) -> T:
        obj = object.__getattribute__(self, "_obj")
        if obj is None:
            factory = object.__getattribute__(self, "_factory")
            obj = factory()
            object.__setattr__(self, "_obj", obj)
        return obj

    def __getattr__(self, name: str):
        # Если атрибут не найден в прокси (а их почти нет), делегируем реальному объекту
        return getattr(self._get(), name)

    def __setattr__(self, name: str, value):
        # Запрещаем создавать/менять произвольные атрибуты на прокси
        if name in ("_factory", "_obj"):
            object.__setattr__(self, name, value)
            return
        # Если объект уже загружен — считаем что хотели изменить поле реального объекта
        obj = object.__getattribute__(self, "_obj")
        if obj is None:
            raise AttributeError(
                f"Cannot set '{name}' on proxy before target is loaded "
                f"(prevents silent typos). Load it first by accessing any attribute."
            )
        setattr(obj, name, value)

    def __repr__(self) -> str:
        obj = object.__getattribute__(self, "_obj")
        return "<SafeLazyProxy (not loaded)>" if obj is None else f"<SafeLazyProxy loaded {obj!r}>"

class User:
    __slots__ = ("id", "name", "role")  # тоже защищаемся от опечаток на самом объекте

    def __init__(self, user_id: int):
        print(f"[DB] Loading user {user_id} ...")
        self.id = user_id
        self.name = "Samir"
        self.role = "admin"

def load_user() -> User:
    return User(42)

u = SafeLazyProxy(load_user)

# u.nmae = "Oops"  # AttributeError: не даст "тихо" создать атрибут (опечатка)

print(u.name)       # триггерит загрузку
u.name = "Samuel"   # теперь меняем уже реальный объект
print(u.name)

# u.some_new_field = 123  # AttributeError или уйдёт в реальный объект, но у него __slots__ => тоже упадёт

[DB] Loading user 42 ...
Samir
Samuel


### 4. Многопоточный сценарий

In [ ]:
from __future__ import annotations
import time

import threading
from typing import Callable, Generic, Optional, TypeVar

T = TypeVar("T")

class ThreadSafeLazyProxy(Generic[T]):
    __slots__ = ("_factory", "_obj", "_lock")

    def __init__(self, factory: Callable[[], T]):
        object.__setattr__(self, "_factory", factory)
        object.__setattr__(self, "_obj", None)
        object.__setattr__(self, "_lock", threading.Lock())

    def _get(self) -> T:
        obj = object.__getattribute__(self, "_obj")
        if obj is not None:
            return obj

        lock = object.__getattribute__(self, "_lock")
        with lock:
            obj = object.__getattribute__(self, "_obj")
            if obj is None:
                factory = object.__getattribute__(self, "_factory")
                obj = factory()
                object.__setattr__(self, "_obj", obj)
            return obj

    def __getattr__(self, name: str):
        return getattr(self._get(), name)

    def __repr__(self) -> str:
        obj = object.__getattribute__(self, "_obj")
        return "<ThreadSafeLazyProxy (not loaded)>" if obj is None else f"<ThreadSafeLazyProxy loaded {obj!r}>"


In [ ]:
class NaiveLazyProxy(Generic[T]):
    __slots__ = ("_factory", "_obj")

    def __init__(self, factory: Callable[[], T]):
        object.__setattr__(self, "_factory", factory)
        object.__setattr__(self, "_obj", None)

    def _get(self) -> T:
        obj = object.__getattribute__(self, "_obj")
        if obj is None:
            factory = object.__getattribute__(self, "_factory")
            obj = factory()                      # <-- гонка: оба потока могут дойти сюда
            object.__setattr__(self, "_obj", obj)
        return obj

    def __getattr__(self, name: str):
        return getattr(self._get(), name)

In [ ]:
init_count = 0
init_lock = threading.Lock()

class ExpensiveClient:
    def __init__(self):
        global init_count
        with init_lock:
            init_count += 1
            n = init_count
        print(f"[INIT #{n}] creating client...")
        time.sleep(0.2)  # имитируем дорогую инициализацию (I/O, TLS, прогрев, etc.)
        self.base_url = "https://api.example.com"

def make_client() -> ExpensiveClient:
    return ExpensiveClient()

client = ThreadSafeLazyProxy(make_client)
print(client.base_url)  # создастся один раз даже при гонках

[INIT #1] creating client...
https://api.example.com


In [ ]:
def factory() -> ExpensiveClient:
    return ExpensiveClient()

def run_race(proxy, title: str):
    global init_count
    init_count = 0

    start = threading.Barrier(3)  # 2 worker-потока + главный поток
    results = []
    res_lock = threading.Lock()

    def worker(idx: int):
        start.wait()  # стартуем одновременно
        url = proxy.base_url  # триггерим загрузку
        with res_lock:
            results.append((idx, url))

    t1 = threading.Thread(target=worker, args=(1,))
    t2 = threading.Thread(target=worker, args=(2,))
    t1.start()
    t2.start()

    start.wait()      # отпускаем оба потока одновременно
    t1.join()
    t2.join()

    print(f"\n=== {title} ===")
    print("results:", sorted(results))
    print("init_count:", init_count)


In [ ]:
run_race(NaiveLazyProxy(factory), "NaiveLazyProxy (ожидаем гонку: init_count может быть 2)")

[INIT #1] creating client...
[INIT #2] creating client...

=== NaiveLazyProxy (ожидаем гонку: init_count может быть 2) ===
results: [(1, 'https://api.example.com'), (2, 'https://api.example.com')]
init_count: 2


In [ ]:
run_race(ThreadSafeLazyProxy(factory), "ThreadSafeLazyProxy (без гонки: init_count должен быть 1)")

[INIT #1] creating client...

=== ThreadSafeLazyProxy (без гонки: init_count должен быть 1) ===
results: [(1, 'https://api.example.com'), (2, 'https://api.example.com')]
init_count: 1


In [ ]:
class Brainfuck:
  def crack(self):
    print("crack")


class GamerGate:
  def crack(self):
    print("crack")

In [ ]:
def crack(obj):
  obj.crack()


In [ ]:
crack(Brainfuck())
crack(GamerGate())

crack
crack


In [ ]:
from collections.abc import Iterable

def is_iterable(a):
  return isinstance(a, Iterable)

In [ ]:
is_iterable([]), is_iterable((1, 2, 3)), is_iterable("abc")

(True, True, True)

In [ ]:
class MyIterable:
  def __init__(self, result: list[int]) -> None:
    self.result = result


  def __iter__(self):
    ...

In [ ]:
is_iterable(MyIterable([1,2,3,4]))

True

In [4]:
from typing import Protocol, runtime_checkable

@runtime_checkable
class Language(Protocol):
  def execute_pgm(pgm: str):
    ...



class Python:
  def execute_pgm(pgm):
    print("pgm")



isinstance(Python, Language)

True